In [11]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [12]:
# 데이터 로드
article_df = pd.read_csv('article.csv')
article_df.tail()

,category,content,link,title
595,105,"[디지털데일리 채수웅기자] 과학기술정보통신부(장관 최기영)는 21일 KT, SK텔레...",https://news.naver.com/main/read.nhn?mode=LSD&...,"과기정통부-통신사, 재난방지 대책 추진현황 점검"
596,105,"게임 기획 단계부터 타격감 위주로 설계(지디넷코리아=김한준 기자)""슬랩잇은 기획 ...",https://news.naver.com/main/read.nhn?mode=LSD&...,STAC 2019 게임 수상작 '슬랩잇' 고등학생 퀄리티 넘어섰다
597,105,"과기정통부, 22일자로 IBS 신임 원장 임명[아이뉴스24 최상국 기자] 기초과학연...",https://news.naver.com/main/read.nhn?mode=LSD&...,"노도영 IBS 신임 원장 임명, ""IBS 개혁 · 중이온가속기 성공"" 임무"
598,105,병역 대체복무제도 개선 확정…중소·중견기업에 배정 늘려(세종=연합뉴스) 신선미 기자...,https://news.naver.com/main/read.nhn?mode=LSD&...,2025년 전문연구요원 석사 300명 감축…박사는 1천명 유지(종합)
599,105,엔지니어들이 SOC실증연구센터에서 스마트 건설 방식으로 공사를 진행하고 있다. 제...,https://news.naver.com/main/read.nhn?mode=LSD&...,"SKT, ‘5G+건설·기계’ 드론 띄워 측량, 굴삭기로 정밀공사 ‘착착’"


In [14]:
# 데이터 전처리 : 데이터 셋 분리
X_train, X_test, y_train, y_test = train_test_split(article_df.content, article_df.category, test_size=0.1, random_state=1)
len(X_train), len(X_test), len(y_train), len(y_test)

(540, 60, 540, 60)

In [18]:
# 모델 만들기
clf = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultinomialNB(alpha=0.1)),
])

In [20]:
# 학습
model = clf.fit(X_train.values.astype('str'), y_train)

In [21]:
# 모델 성능 확인
y_pred = model.predict(X_test)

In [22]:
list(y_test[:5]), list(y_pred[:5])

([104, 104, 105, 104, 102], [104, 104, 105, 104, 102])

In [23]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         100       0.60      0.90      0.72        10
         101       0.56      0.62      0.59         8
         102       0.43      0.30      0.35        10
         103       1.00      0.29      0.44         7
         104       1.00      0.81      0.90        16
         105       0.64      1.00      0.78         9

    accuracy                           0.68        60
   macro avg       0.70      0.65      0.63        60
weighted avg       0.73      0.68      0.67        60



### 모델 사용하기

In [24]:
categories ={
    100: "정치" ,
    101: "경제" ,
    102: "사회" ,
    103: "생활/문화",
    104: "세계",
    105: "IT/과학",
}

In [27]:
contents = [
    "네이버와 카카오는 드론 기술 발전에 주목",
    "강남 3구 부동산 정책 실패 요인?",
    "네이버 주식 상승률 역대 최고",
]

In [29]:
datas = {
    "content" : contents,
    "category_code" : model.predict(contents),
}
df = pd.DataFrame(datas)
df

,content,category_code
0,네이버와 카카오는 드론 기술 발전에 주목,105
1,강남 3구 부동산 정책 실패 요인?,101
2,네이버 주식 상승률 역대 최고,101


In [33]:
## 각 카테고리에 해당할 확률을 보여줌
model.predict_proba([content[0]])

array([[0.06394595, 0.11837843, 0.13189608, 0.10058136, 0.08305473,
        0.50214345]])

In [34]:
df["category"] = df["category_code"].apply(lambda data : categories[data])
df["proba"] = df["content"].apply(lambda data : round(max(model.predict_proba([data])[0]), 2))
df

,content,category_code,category,proba
0,네이버와 카카오는 드론 기술 발전에 주목,105,IT/과학,0.50
1,강남 3구 부동산 정책 실패 요인?,101,경제,0.34
2,네이버 주식 상승률 역대 최고,101,경제,0.24


In [35]:
# 모델 저장
pickle.dump(model, open("clf.pkl", "wb"))